In [1]:
import numpy as np
import pandas as pd
from custom_func import *

In [2]:
df_P04_005 = pd.read_excel(f"{INPUTS_PATH}/P4/P04_005.xlsx", dtype={"КОАТУУ населеного пункту": str})
settlement = pd.read_excel(f"{INPUTS_PATH}/P99/settlements.xlsx")

In [3]:
urban = ["місто", "селище міського типу"]
rural = ["село",  "селище"]

In [4]:
df_P04_005["param_type"] = np.select(
    condlist=[
        df_P04_005["Тип населеного пункту (село, селище, селище міського типу, місто)"].isin(urban),
        df_P04_005["Тип населеного пункту (село, селище, селище міського типу, місто)"].isin(rural)
    ],
    choicelist=["p4_02_raw", "p4_03_raw"],
    default= np.nan
)

In [6]:
df_P04_005_fin = df_P04_005.loc[
    (df_P04_005["Дата прийняття рішення про затвердження Генерального плану населеного пункту"] <= "2020-06-30") &
    (df_P04_005["Дата прийняття рішення про затвердження Генерального плану населеного пункту"] >= "2020-04-01")
]

In [7]:
df_P04_005_fin = df_P04_005.groupby(["region", "param_type"])["Населений пункт"].size().reset_index()
df_P04_005_fin = df_P04_005_fin.pivot(index='region', columns='param_type').fillna(0).reset_index()
df_P04_005_fin.columns = df_P04_005_fin.columns.droplevel()
df_P04_005_fin.columns = ["region", "p4_02_raw", "p4_03_raw"]

In [9]:
df_P04_005_fin = pd.merge(
    settlement.rename(columns={"Кількість населених пунктів": "total"}),
    df_P04_005_fin,
    how="outer"
)

df_P04_005_fin["p4_02_raw"] = df_P04_005_fin["p4_02_raw"] / df_P04_005_fin["total"]
df_P04_005_fin["p4_03_raw"] = df_P04_005_fin["p4_03_raw"] / df_P04_005_fin["total"]

In [10]:
normalize_parameter(df_P04_005_fin,'p4_02_raw','p4_02',min_bound=0,max_bound=1)
normalize_parameter(df_P04_005_fin,'p4_03_raw','p4_03',min_bound=0,max_bound=1)

Емпіричні границі: (0, 1)
Коефіцієнти нормалізації: (1.0, -0.0)
Параметр p4_02 нормалізовано і додано до таблиці

Емпіричні границі: (0, 1)
Коефіцієнти нормалізації: (1.0, -0.0)
Параметр p4_03 нормалізовано і додано до таблиці



In [11]:
df_P04_006 = pd.read_excel(f"{INPUTS_PATH}/P4/P04_006.xlsx")
roads = pd.read_csv(f"{INPUTS_PATH}/P99/all_roads.csv").rename(columns={"Область": "region"})

In [12]:
df_P04_006 = df_P04_006.loc[pd.to_datetime(df_P04_006["Дата початку ремонтних робіт"]) < "2020-07-01"]

df_P04_006_fin = pd.concat([
    roads.groupby("region")["Довжина (протяжність) дороги (км)"].sum().rename('roads_all'),
    df_P04_006.groupby('region')['Довжина (протяжність) відремонтованої ділянки (км)'].sum().rename('roads_repaired'),
    df_P04_006.groupby('region')['Загальна вартість ремонту (грн)'].sum().rename('price')
], axis=1)

df_P04_006_fin['price_km'] = df_P04_006_fin['price']/df_P04_006_fin['roads_repaired']
df_P04_006_fin['p4_05_raw'] = df_P04_006_fin['roads_repaired']/df_P04_006_fin['roads_all']
df_P04_006_fin = df_P04_006_fin.reset_index().rename(columns={"index": "region"})
normalize_parameter(df_P04_006_fin,'p4_05_raw','p4_05',min_bound=0)

Емпіричні границі: (0, 0.020276599459073542)
Коефіцієнти нормалізації: (49.31793430246567, -0.0)
Параметр p4_05 нормалізовано і додано до таблиці



In [13]:
#Група 1: p4_01, сміття
#Група 2: p4_02-p4_03, генплани
#Група 3: p4_05-p4_06, дороги

dict_weights = {
    'p4_01':1,
    'p4_02':1,
    'p4_03':1,
    'p4_05':1,
    'p4_06':1
}

sources = [df_P04_005_fin, df_P04_006_fin]
save_data(sources,regions,dict_weights,'P4')

Дані за галузевим параметром P4 збережено
               region     p4_02     p4_03     p4_05        P4
0           Волинська  0.000000  0.002778  0.526775  1.059105
1           Вінницька  0.000665  0.008644  0.318542  0.655701
2    Дніпропетровська  0.000000  0.000666  0.948457  1.898247
3            Донецька  0.002356  0.000000  0.276283  0.557277
4         Житомирська  0.000601  0.002405  0.377780  0.761574
5        Закарпатська  0.000000  0.006568  0.422385  0.857906
6          Запорізька  0.001054  0.028451  0.362038  0.783085
7            Київська  0.001693  0.011854  0.434851  0.896798
8   Івано-Франківська  0.001362  0.004087  1.000000  2.010899
9      Кіровоградська  0.002058  0.012346  0.204456  0.437718
10          Луганська  0.000000  0.001842  0.653571  1.310825
11          Львівська  0.000000  0.005176  0.341205  0.692762
12       Миколаївська  0.001098  0.010977  0.133177  0.290503
13            Одеська  0.003401  0.005952  0.410382  0.839471
14         Полтавська  0.000